# FAIRHealth PHT PPDML

In [1]:
import requests
import json
import uuid
import time
from IPython.display import Image
from IPython.core.display import HTML 

import ipywidgets as widgets
from ipywidgets import interact, interactive, interact_manual, Layout, Button

In [2]:
signalStation = "http://vwdata.personalhealthtrain.net"

def getStations(signalStation):
    response = requests.get(signalStation + "/client")
    return json.loads(response.text)

def defineTrain(image, inputVars, runId):
    return {
        "image": image,
        "inputString": ("%s" % json.dumps(inputVars)),
        "runId": runId
    }

def requestExecution(signalStation, clientId, taskDescription):
    response = requests.post(signalStation + "/client/" + str(clientId) + "/task/add", json=taskDescription)
    taskId = json.loads(response.text)["taskId"]
    return taskId

def getExecutionResult(signalStation, clientId, taskId, wait=True):
    url = signalStation + "/client/" + str(clientId) + "/task/" + str(taskId) + "/result"
    print(url)
    jsonResult = json.loads(requests.get(url).text)
    
    while len(jsonResult)==0 & wait:
        time.sleep(2)
        jsonResult = json.loads(requests.get(url).text)
    
    return jsonResult

## Set up for two data stations (two parties)

In [3]:
Image(url= "https://github.com/sunchang0124/PHTDemon/blob/master/Pics/demonPic2.png?raw=true", width=600, height=600)

In [4]:
getStations(signalStation)

[{'country': 'The Netherlands',
  'email': 'johan.vansoest@maastro.nl',
  'id': 1,
  'institute': 'Maastricht University',
  'ip': '192.168.96.1',
  'last_seen': '2019-05-28 15:09:30.599025',
  'name': 'The Maastricht Study'},
 {'country': 'Netherlands',
  'email': 'johan.vansoest@maastro.nl',
  'id': 2,
  'institute': 'MAASTRO Clinic',
  'ip': '192.168.96.1',
  'last_seen': '2019-05-28 15:09:31.622865',
  'name': 'VWData Station Maastro'}]

## This is data party B

In [5]:
def stage_B1(event):
    
    results = getExecutionResult(signalStation, input_clientID.value, input_taskID.value)
    result = json.loads(results[0]["response"])

    inputB = input_data_B.value.split(';')
    columnsNum = len(inputB)
    item = []
    for i in range(0, columnsNum):
        item.append([int(i) for i in inputB[0].split(',')])

    
    result["stage"] = 2
    result["dataB"] = item
    mySecondTrain = defineTrain("sophia921025/ppdml_demon", result, 1)

    clientId = 2
    taskId = requestExecution(signalStation, clientId, mySecondTrain)
    print("Task submitted with ID: %s" % taskId)

    resultsSecond = getExecutionResult(signalStation, clientId, taskId)
    print(resultsSecond[0]["log"])
    resultSecond = json.loads(resultsSecond[0]["response"])
    print(resultSecond.keys())

    display(Image(url= "https://github.com/sunchang0124/PHTDemon/blob/master/Pics/demonPic7.png?raw=true", width=600, height=600))
    display(Image(url= "https://github.com/sunchang0124/PHTDemon/blob/master/Pics/demonPic8.png?raw=true", width=600, height=600))
    display(Image(url= "https://github.com/sunchang0124/PHTDemon/blob/master/Pics/demonPic9.png?raw=true", width=600, height=600))

    sumNoisesAB = resultSecond["sumNoisesAB"]
    event.value = sumNoisesAB


In [6]:
#### Demon: Give your data here: ####
style = {'description_width': 'initial'}
uniLayout = Layout(width='50%', height='30px')

input_clientID = widgets.BoundedIntText(value=1,max=100,description='Retrieve intermediate result from Client ID:',disabled=False, style=style, layout=uniLayout)
input_taskID = widgets.BoundedIntText(value=150,max=10000,description='Retrieve intermediate result from Task ID:',disabled=False, style=style, layout=uniLayout)
input_data_B = widgets.Text(value='1,2,3,2,1;2,2,1,4,3', description='Input data B (comma separate): ', style=style, layout=uniLayout)
button1 = Button(description="Run", button_style='success', value=None)

button1.on_click(stage_B1)
display(input_clientID, input_taskID, input_data_B, button1)

BoundedIntText(value=1, description='Retrieve intermediate result from Client ID:', layout=Layout(height='30px…

BoundedIntText(value=150, description='Retrieve intermediate result from Task ID:', layout=Layout(height='30px…

Text(value='', description='Input data B (comma separate): ', layout=Layout(height='30px', width='50%'), style…

Button(button_style='success', description='Run', style=ButtonStyle())

http://vwdata.personalhealthtrain.net/client/1/task/149/result
Task submitted with ID: 150
http://vwdata.personalhealthtrain.net/client/2/task/150/result
Stage: 2


=======================INTERNAL FILE LOG===================== 

dict_keys(['sumNoisesAB', 'sumNoisesB'])


In [ ]:
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -

## Calculate final outcome

In [7]:
def stage_B2(event2):
    results = getExecutionResult(signalStation, input_clientID.value, input_taskID.value)
    resultThird = json.loads(results[0]["response"])

    resultThird["stage"] = 4
    resultThird["sumNoisesAB"] = sumNoisesAB
    
    myFourthTrain = defineTrain("sophia921025/fairhealth_test", resultThird, 1)
    print(resultThird.keys())

    clientId = 2
    taskId = requestExecution(signalStation, clientId, myFourthTrain)
    print("Task submitted with ID: %s" % taskId)

    resultsFourth = getExecutionResult(signalStation, clientId, taskId)
    print(resultsFourth[0]["log"])
    resultFourth = json.loads(resultsFourth[0]["response"])

In [9]:
sumNoisesAB = button1.value

input_clientID = widgets.BoundedIntText(value=1,max=100,description='Retrieve intermediate result from Client ID:',disabled=False, style=style, layout=uniLayout)
input_taskID = widgets.BoundedIntText(value=151,max=10000,description='Retrieve intermediate result from Task ID:',disabled=False, style=style, layout=uniLayout)
button2 = Button(description="Run", button_style='success', value=None)

button2.on_click(stage_B2)
display(input_clientID, input_taskID, button2)

BoundedIntText(value=1, description='Retrieve intermediate result from Client ID:', layout=Layout(height='30px…

BoundedIntText(value=151, description='Retrieve intermediate result from Task ID:', layout=Layout(height='30px…

Button(button_style='success', description='Run', style=ButtonStyle())

http://vwdata.personalhealthtrain.net/client/1/task/151/result
dict_keys(['randomsSumSet', 'sumNoisesBARand', 'XaTXa', 'stage', 'sumNoisesAB'])
Task submitted with ID: 152
http://vwdata.personalhealthtrain.net/client/2/task/152/result
Stage: 4
XaTXa: 10 - 10
XbTXa: 10 - 1
XaTXb: 4 - 1
XbTXb: 5 - 1
XbTXb_exclY: 4 - 1
Coefficients: 
 [[-1.15678321e+01]
 [-4.89996193e+00]
 [-6.35477806e+01]
 [-2.85351329e+00]
 [ 2.17420659e-01]
 [ 3.14462563e+00]
 [ 1.02737671e+02]
 [-1.13506411e+01]
 [-3.14968709e+02]
 [-2.84356047e+02]
 [ 1.00324224e+02]
 [ 7.45891879e-01]
 [ 1.47713293e-02]]
Intercept:  -9.219794829436864


=======================INTERNAL FILE LOG===================== 

